# 08. CLONE DETECTION

1. Introduction
2. Approaches
3. LLMs
4. Exercise
5. References

# 1. Введение

#### DEFINITION

Клоны — это фрагменты кода, реализующие одну и ту же функциональность.

Связь синтаксиса и семантики:
- identical syntax => identical semantics
- regular code: different syntax, different semantics
- anti-patterns, bugs: similar syntax & different semantics
- clones: identical semantics

Часто возникают в результате "copy & paste".

#### TYPES

**Type-1 (Exact clones)**

Идентичные фрагменты кода, за исключением различий в пробелах, комментариях и вёрстке.

**Type-2 (Renamed/Parameterized)**

Идентичные фрагменты кода, за исключением различий в именах идентификаторов и литеральных значениях, в дополнение к различиям клонов типа 1.

**Type-3 (Near miss clones/Gapped clones)**

Синтаксически похожие фрагменты кода, которые содержат добавленные, измененные и/или удаленные операторы кода, в дополнение к различиям клонов типа 1 и типа 2.

**Type-4 (Semantic clones)**

Синтаксически разные фрагменты кода, но семантически схожие с точки зрения функционала.


> Примеры:
> ![types](./res/08_clone_types.png)
>
> ![types](./res/08_equivalent_transformations.png)

#### ЗАЧЕМ УМЕТЬ НАХОДИТЬ КЛОНЫ

Несмотря на то, что клонирование может быть полезным во многих отношениях, оно также может быть вредным при обслуживании и развитии программного обеспечения.

Например, если в каком-то фрагменте кода обнаружена ошибка, то все клоны должны быть проверены на наличие такой же ошибки. Дублированные фрагменты также могут значительно увеличить объем работы при улучшении или адаптации кода.


Кроме того, дедупликация (исключение клонов) --- стандартный процесс при подготовке датасетов.

#### СКОЛЬКО ВСЕГО КЛОНОВ

Повторное использование фрагментов кода путем копирования и вставки с незначительной адаптацией или без нее — обычное дело при разработке программного обеспечения.

В результате программные системы часто содержат клоны. Исследования показывают, что значительная часть (от 7% до 23%) кода в типичной программной системе была клонирована.

- Linux: 22.3% ([Sheneamer Kalita, 2016](https://www.ijcaonline.org/archives/volume137/number10/24308-2016908896))
- JDK: 29% ([Kamiya et al, 2002](https://www.cs.drexel.edu/~spiros/teaching/CS675/papers/clone-kamiya.pdf))

70% кода на GitHub состоит из клонов ранее созданных файлов.

#### КАРТА ДУБЛИРОВАНИЯ КОДА ([Lopes et al, 2017](http://janvitek.org/pubs/oopsla17b.pdf)):

- ось X: количество файлов в проекте
- ось Y: количество коммитов на проект

Число на каждом квадрате --- это процент дублированных файлов для всех проектов этого квадрата.

![dejavu](./res/08_dejavu_tiles_1.png)
![dejavu](./res/08_dejavu_tiles_2.png)

#### DATASETS

- [BigCloneBench](https://github.com/clonebench/BigCloneBench) --- включает более 8 миллионов клонов из 25000 Java-репозиториев
- [BigCloneEval](https://github.com/jeffsvajlenko/BigCloneEval) --- фреймворк для тестирования разных методов по детекции клонов

# 2. Approaches

#### TEXT-BASED

Например, на основе сравнения строк.

1. нормализуем код
2. для каждой строки вычисляем значение некоторой хэш-функции
3. визуализируем в виде матрицы: если две строки имеют одинаковое значение хэш-функции, то пишем 1, иначе 0
4. выявляем паттерны (например, диагональные последовательности) из единиц, они соответствуют повторяющемуся коду

> Пример:
>
> ([Ducasse et al, 1999](https://ieeexplore.ieee.org/document/792593))
> 
> ![](./res/08_clones_text.png)

#### TOKEN-BASED

Вместо сравнения символов используется сравнение токенов.

В случае подходов на основе токенов каждая строка исходного кода делится на токены в соответствии с лексическими правилами интересующего языка программирования. Токены вместе образуют последовательность токенов, используемую для сравнения. Все пробелы (включая разрывы строк и табуляции) и комментарии между токенами удаляются из последовательностей токенов.

Например, если фрагменты кода отличаются только названиями переменных, то разные названия будут соответствовать одному и тому же токены и, поэтому, не помешают при сравнении.

Например:
- [CCFinder](https://www.cs.drexel.edu/~spiros/teaching/CS675/papers/clone-kamiya.pdf)
- [CCFinderX](https://github.com/gpoo/ccfinderx)

#### AST-BASED

Сравнение происходит на основе абстрактных синтаксических деревьев.

Например, так:
1. строится абстрактное синтаксическое дерево
2. выделяются поддеревья
3. вычисляются значения некоторой хэш-функции для этих поддеревьев
4. если два поддерева получили одинаково значение, то происходит дополнительное сравнение двух деревьев

![](https://leanovate.github.io/bedcon/talk/abstract_syntax_tree.png)

#### DATA FLOW BASED

Методы нахождения клонов на основе токенов и методы на основе синтаксиса зависят от порядка в котором написаны инструкции программы.
Если разработчики меняют в скопированном коде инструкции местами, то скопированный код не будет найден в качестве клона.

Тем не менее, порядок не может быть изменен произвольно без изменения смысла программы.
Граф программных зависимостей представляет собой представление программы, которое представляет только зависимость управления и данных между операторами.

![](https://www.researchgate.net/profile/Sergey-Troshin/publication/358740900/figure/fig1/AS:11431281080328895@1661254252291/Example-of-Data-Flow-Graph-from-GraphCodeBERT-Guo-et-al-2021_W640.jpg)

Клоны могут быть идентифицированы как изоморфные подграфы в графе программных зависимостей.

Задача является NP-сложной, используются приближённые алгоритмы.

> Пример изоморфных графов
> 
> ![](https://static.javatpoint.com/tutorial/dms/images/graph-isomorphism-in-discrete-mathematics.png)

#### IMAGE-BASED

[Ragkhitwetsagul et al - A picture is worth a thousand words Code clone detection based on image similarity 2018](https://ieeexplore.ieee.org/document/8327318)

![scheme](./res/08_clone_detection_image_scheme.png)

Подход:

1. 
   - распарсить код и извлечь методы
   - удалить комментарии
   - красиво напечатать
   - преобразовать в html с подсветкой синтаксиса
2. 
   - конвертировать каждый метод в PNG-изображение
   - получить RGB-изображение
3. 
   - преобразовать изображение в негативное изображение
   - применить фильтры (гауссовый фильтр, например)
4. 
   - сравнить два изображения. Можно использовать **The Earth Mover’s Distance (EMD)** --- метрику, которая рассматривает сравнение изображений как транспортную задачу (задача оптимального транспорта), находя минимальную стоимость преобразования одного распределения в другое
  
![](./res/08_clone_detection_image_1.png)
![](./res/08_clone_detection_image_2.png)
![](./res/08_clone_detection_image_3.png)

#### EMBEDDING-BASED

Происходит сравнение векторов в метрическом пространстве. Вектора строятся на основе различных метрик или предобученных моделей (BERT, RoBERTa, CodeBERT, GraphCodeBERT, ...).

**Contrastive learning** --- это подход в машинном обучении, для выявления похожих и непохожих объектов.
Модель обучается строить эмбеддинги для объектов таким образом, что похожие объекты будут иметь близкие вектора, а разные объекты, соответственно, будут различаться.
Поэтому решение о том, являются ли две программы клонами или нет, принимается на основе расстояний между двумя векторами.

![](./res/08_contrastive_learning_pipeline.png)

- $o$, $o'$ --- two different objects
- $f$ --- an encoder model
- $q$, $k$ --- embeddings of the obejcts
- $g$ --- a special projector model which is specific for algorithm (in a simple case, $g$ can be an identical function, i.e., just pass $q$ and $k$ onwards
- $q'$, $k'$ --- transformed vectors for the loss function $L$

**Подробнее**
- [Evaluation of Contrastive Learning with Various Code Representations for Code Clone Detection](https://arxiv.org/abs/2206.08726)

# 3. LLMs

[Towards Understanding the Capability of Large Language Models on Code Clone Detection: A Survey](https://arxiv.org/abs/2308.01191)

Clones:

Additional sub-categories for Type-3 and Type-4 clones based on their syntactical similarity scores:
- Very Strongly Type-3 (VST3) clones, with similarity scores in the range of [0.9, 1.0)
- Strongly Type-3 (ST3) clones, with similarity scores between [0.7, 0.9)
- Moderately Type-3 (MT3) clones, with similarity scores between [0.5, 0.7)
- Weakly Type-3/Type-4 (WT3/T4) clones, with similarity scores between [0.0, 0.5)

Instructions:

![](res/08_prompts.png)

#### RQ1: Can LLMs detect code clones with a simple prompt?

![](res/08_llms_vs_non-llms.png)


> Using open-source LLMs for clone detection yields superior results in identifying Type-3 and Type-4 clone pairs when relying solely on a simple prompt. However, it does exhibit slightly poorer performance when detecting Type-1 and Type-2 clone pairs compared to existing tools. Notably, GPT-3.5-Turbo and GPT-4 stand out with the highest recall and accuracy rates across nearly all clone types.

#### RQ2: How do LLMs perform by using one-step chain-of-thought prompts?

* In one-step prompt engineering, the model is tasked with detecting code clones from various perspectives (i.e. clone type, similarity, and analogous lines of code pair).
* In multi-step prompt engineering, the model initially analyzes each function from multiple perspectives, subsequently integrating all the intermediate reasonings. This approach enables the model to detect code clones with prior knowledge, rather than merely following human instructions to provide a binary "yes" or "no" response.


![](res/08_clone_type_reasoning.png)

![](res/08_similar_line_reasoning.png)

> The clone detection performance of GPT-3.5-Turbo and GPT-4 can be improved by requiring models to provide clone type, similarity, reasoning, and similarity lines. Using one-step chain-of-thought prompts allows the models to analyze code pairs and intermediate reasoning, leading to better clone detection.

#### RQ3: Can LLMs perform better by using multi-step chain-of-thought prompts?

![](res/08_separate_explanation_codes.png)

> The clone detection performance of GPT-3.5-Turbo and GPT-4 can be improved by Multi-Step Chain-of-Thought prompts, including separating explanations and codes. Different from RQ2, separating explanations provide models of independent intermediate reasoning of code, and separating codes provide models of independent explanation of code, which avoid the influences between generated information.

#### RQ4: How do LLMs perform using code embedding?

This question focuses on whether LLMs can provide superior results compared to traditional pre-trained language models through code compression. 

![](res/08_similarity_distribution.png)

> Text-embedding-ada-002 is more effective than specialized CodeBERT models in identifying cloned code, exhibiting superior overall performance. The advantage of Text-embedding-ada-002 lies in its capacity to generate a wider range of similarity scores, leading to better discrimination between true and false positives.

#### RQ5: How does the performance of LLMs in code clone detection vary across different programming languages?

![](res/08_different_languages.png)

> The performance of LLMs in code clone detection varies across different programming languages, with a trend of superior results in Python, likely due to its inherent simplicity and prevalence in training data

# 4. Exercise

Сравнить работу предобученных моделей (например, CodeBERT, GraphCodeBERT) и LLM в задаче детекции клонов. Сделать выводы.

# 5. References

- Allamanis - The adverse effects of code duplication in machine learning models of code
- Baker - On finding duplication and near-duplication in large software systems
- Bogomolov et al - Sosed A tool for finding similar software projects
- [Code code analysis](https://link.springer.com/book/10.1007/978-981-16-1927-4)
- Ducasse et al - A language independent approach for detecting duplicated code
- Gupta Gupta - Literature survey of clone detection techniques
- Huang et al - Code clone detection based on event embedding and event dependency
- Ivanov et al - AntiCopyPaster Extracting code duplicates as soon as they are introduced in the IDE
- Khajezade et al - Evaluating few shot and contrastive learning methods for code clone detection
- Kim et al - Revisiting binary code similarity analysis using interpretable feature engineering and lessons learned
- Koschke - Survey of research on software clones
- Lopes et al - DejaVu A map of code duplicates on GitHub
- Nadim et al - Evaluating the performance of clone detection tools in detecting cloned co-change candidates
- Ragkhitwetsagul - Code similarity and clone search in large-scale source code data
- Ragkhitwetsagul et al - A picture is worth a thousand words Code clone detection based on image similarity
- Rahman et al - Clone detection on large Scala codebases
- Roy et al - Comparison and evaluation of code clone detection techniques and tools
- Saini et al - Oreo Detection of clones in the twilight zone
- Sheneamer Kalita - A survey of software clone detection techniques
- [Svajlenko Roy - A survey on the evaluation of clone detection performance and benchmarking](https://arxiv.org/abs/2006.15682)
- Svajlenko Roy - Efficiently measuring an accurate and generalized clone detection precision using clone clustering
- White et al - Deep learning code fragments for code clone detection
- Yahay Kim - Cross-languages source code clone detection using deep learning with InferCode
- Zhang et al - Challenging machine learning-based clone detectors via semantic-preserving code transformations
- Zhang et al - The development and prospect of code clone
- Zubkov et al - Evaluation of contrastive learning with various code representations for code clone detection 2022